In [160]:
# import relevant libraries
from bs4 import BeautifulSoup
import requests
import json
import os
import time

import pandas as pd
import regex as re
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

# get latest Chrome driver and maximize screen
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

C:\Users\bartb\AppData\Local\Temp\ipykernel_28360\555813881.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [161]:
# Get the chromedriver.
PATH = 'C:/Users/bartb/Downloads/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(PATH)

C:\Users\bartb\AppData\Local\Temp\ipykernel_28360\3124890759.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [162]:
# Go to the Tripadvisor website for London England.
# And because we're scraping english restaurants, go to co.uk site
url = 'https://www.tripadvisor.co.uk/Restaurants-g186337-Liverpool_Merseyside_England.html'
driver.get(url)
time.sleep(3)
# Click OK for cookies if present
try:
    driver.find_element(By.ID, "onetrust-accept-btn-handler").click()
except:
    pass
time.sleep(2)
# Divide the amount of restaurants by 30, round upwards. This is the amount of restaurants it will be scraping.
pages = int(int(driver.find_element(By.XPATH,"*//span[@class = 'b']").text)/30)
pages

49

In [129]:
%%time
i = 1
links = []
while i <= pages:
    html = driver.page_source # get the page source
    soup = BeautifulSoup(html,'html.parser') # transform to BeautifulSoup object
    links.append(soup.find_all(class_='Lwqic Cj b')[1]['href']) #get the first non-ad restaurant link (top/bottom are ads)
    time.sleep(1)
        
    # Scroll to the element, otherwise it might fail
    target = driver.find_element(By.XPATH,"*//a[@class = 'nav next rndBtn ui_button primary taLnk']")
    actions = ActionChains(driver)
    actions.move_to_element(target)
    actions.perform()
    time.sleep(1)
    # If its the last page, the button won't be there, thus fail.
    try:    
        target.click()
        time.sleep(3)
    except:
        break
    i=i+1
links

Wall time: 4min 58s


['/Restaurant_Review-g186337-d25107593-Reviews-Rudy_s_Albert_Dock-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d10480792-Reviews-Chamber_36_City_Centre-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d14100376-Reviews-Maggie_Fu-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d1315174-Reviews-Elif-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d8663805-Reviews-Moose_Coffee-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d10485568-Reviews-Jam_Restaurant-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d9758180-Reviews-The_Club_House_L1_Bar_Restaurant-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d13278754-Reviews-Lock_and_Key_Hotel_Bar_Kitchen-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d21235870-Reviews-Greenchillies_South_Indian_Restaurant-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d21084989-Reviews-The_Vale_Bar_Mo_s_Bist

In [163]:
%%time
review_all = pd.DataFrame()

# If this code is executed later the cookies might need to be clicked
try:
    driver.find_element(By.ID, "onetrust-accept-btn-handler").click()
except:
    pass

# For all extracted links get their reviews.
for j in links: #Set less if desired
    driver.get('https://www.tripadvisor.co.uk'+j) # Add the links to the tripadvisor url.
    time.sleep(2)
    driver.find_element(By.ID,"filters_detail_language_filterLang_en").click() # We only use English language reviews
    time.sleep(2)
            
    review_master = get_reviews(driver) # Scrape the reviews, self-built function see below.
    review_all = pd.concat([review_all, review_master])
review_all = review_all.reset_index(drop = True)

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15 is last
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=109.0.5414.75)
Stacktrace:
Backtrace:
	(No symbol) [0x00BC6643]
	(No symbol) [0x00B5BE21]
	(No symbol) [0x00A5DA9D]
	(No symbol) [0x00A609E4]
	(No symbol) [0x00A608AD]
	(No symbol) [0x00A612E5]
	(No symbol) [0x00AC79F7]
	(No symbol) [0x00AAFD7C]
	(No symbol) [0x00AC6B09]
	(No symbol) [0x00AAFB76]
	(No symbol) [0x00A849C1]
	(No symbol) [0x00A85E5D]
	GetHandleVerifier [0x00E3A142+2497106]
	GetHandleVerifier [0x00E685D3+2686691]
	GetHandleVerifier [0x00E6BB9C+2700460]
	GetHandleVerifier [0x00C73B10+635936]
	(No symbol) [0x00B64A1F]
	(No symbol) [0x00B6A418]
	(No symbol) [0x00B6A505]
	(No symbol) [0x00B7508B]
	BaseThreadInitThunk [0x75736BD9+25]
	RtlGetFullPathName_UEx [0x77958FD2+1218]
	RtlGetFullPathName_UEx [0x77958F9D+1165]


In [165]:
len(review_all['restaurant'].unique())

28

In [166]:
review_all['restaurant'].unique()

array(["Rudy's Albert Dock", 'Chamber 36 - City Centre', 'Maggie Fu',
       'Elif', 'Moose Coffee', 'Jam Restaurant',
       'The Club House L1 Bar & Restaurant',
       'Lock and Key Hotel - Bar & Kitchen',
       'Greenchillies South Indian Restaurant',
       "The Vale Bar & Mo's Bistro", 'The Refinery', 'The Brookhouse',
       'Vietnom', 'Barnacle', 'The Black Horse', 'Real Taste Restaurant',
       'Sunshine Tearoom', 'Ten Streets Social', 'Coopers Town House',
       'Que Pasa Cantina', "One O'clock Gun", 'Crumbs Sandwich Shop',
       'La Parilla', 'The Saddle Inn Pub.', "Monty's Cafe", 'Subway',
       'Turpins Pub', 'Mb Cafe 62'], dtype=object)

In [167]:
# Rough categorization of cuisines plus if its a bar/pub or not
df = review_all
df['Italian'] = df['category'].str.contains('Italian|Pizza|Pasta|Lasagna')
df['East Asian'] = df['category'].str.contains('Chinese|Japanese|Sushi|Korean|Taiwanese')
df['South Asian'] = df['category'].str.contains('South Asian|Indian|Bangladeshi|Tandoori|Curry|Pakistani|Sri Lankan|Nepali')
df['South East Asian'] = df['category'].str.contains('South East Asian|Thai|Vietnamese|Malaysian|Indonesian|Filipino|Burmese')
df['American'] = df['category'].str.contains('Burger|American|Canadian|Hotdogs|Barbecue|Poke Bowl|Hawaiian')
df['Middle Eastern'] = df['category'].str.contains('Turkish|Kebab|Falafel|Halal|Middle Eastern|Lebanese|Israeli|Uzbek|Iraqi|Syrian|Egyptian|Iranian|Afghani')
df['Southern European'] = df['category'].str.contains('Spanish|Greek|Portuguese|Bulgarian|Balkan|Paella|Gyros|Mediterranean')
df['FrenchEuropean'] = df['category'].str.contains('French|Bistro|European|German|Dutch|Belgian|Scandinavian|Eastern European|Russian|Polish|Hungarian|Austrian|Swiss')
df['BritishIrish'] = df['category'].str.contains('British|Irish|Fish and Chips')
df['Latin American'] = df['category'].str.contains('Mexican|Burritos|Tacos|Nachos|Brazilian|Caribbean|Argentinian|Peruvian|South American|Chilean')
df['Bar'] = df['category'].str.contains('Wine|Bar|bars|Drinks|Beer|Tea|Coffee|Pub|Cafe')
df['African'] = df['category'].str.contains('African|Ethiopian|Nigerian|Kenyan|Algerian|Moroccan|Tunisian|Angolan|Senegalese')
df

,title,text,review date,rating,restaurant,location,total reviews,1-star,2-star,3-star,...,South Asian,South East Asian,American,Middle Eastern,Southern European,FrenchEuropean,BritishIrish,Latin American,Bar,African
0,Holiday,very busy but sorry to say the mens toliets we...,22 January 2023,3.0,Rudy's Albert Dock,"Unit 6, Britannia Pavillion, Liverpool L3 4AD ...",218,0,3,5,...,False,False,False,False,False,False,False,False,False,False
1,Delicious,To be honest I’m not a pizza lover but my husb...,22 January 2023,5.0,Rudy's Albert Dock,"Unit 6, Britannia Pavillion, Liverpool L3 4AD ...",218,0,3,5,...,False,False,False,False,False,False,False,False,False,False
2,Great pizza great value great service,"Amazing pizzas, with lots of choice. Excellenc...",22 January 2023,5.0,Rudy's Albert Dock,"Unit 6, Britannia Pavillion, Liverpool L3 4AD ...",218,0,3,5,...,False,False,False,False,False,False,False,False,False,False
3,Was so looking forward to it but........,Booked. Arrived. Nice place in Albert Dock. Re...,22 January 2023,2.0,Rudy's Albert Dock,"Unit 6, Britannia Pavillion, Liverpool L3 4AD ...",218,0,3,5,...,False,False,False,False,False,False,False,False,False,False
4,1st time visit to Rudy's Albert Dock,"Great location, great service, great atmospher...",21 January 2023,5.0,Rudy's Albert Dock,"Unit 6, Britannia Pavillion, Liverpool L3 4AD ...",218,0,3,5,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Disappointed,After looking forward to a bite to eat with fr...,12 January 2023,1.0,Mb Cafe 62,"62 Muirhead Avenue East, Liverpool L11 1EL Eng...",5,1,0,0,...,False,False,False,True,True,False,True,False,False,False
1,Great breakfast,This is the second time we have been and the f...,2 January 2023,5.0,Mb Cafe 62,"62 Muirhead Avenue East, Liverpool L11 1EL Eng...",5,1,0,0,...,False,False,False,True,True,False,True,False,False,False
2,First try of a local gem,"We visited today, it was busy with lots of dif...",30 December 2022,5.0,Mb Cafe 62,"62 Muirhead Avenue East, Liverpool L11 1EL Eng...",5,1,0,0,...,False,False,False,True,True,False,True,False,False,False
3,Absolutely worth a visit,I’ve been several times now - for lunch as wel...,17 September 2022,5.0,Mb Cafe 62,"62 Muirhead Avenue East, Liverpool L11 1EL Eng...",5,1,0,0,...,False,False,False,True,True,False,True,False,False,False


In [168]:
# Export the scraped data to csv
df.to_csv('Tripadvisor_reviews_Liverpool.csv')

In [169]:
# Cuisine to one column

df['cuisine'] = np.where(df['Italian'] == True,'Italian',
        np.where(df['East Asian'] == True,'East Asian',
        np.where(df['South Asian'] == True,'South Asian',
        np.where(df['South East Asian'] == True,'South East Asian',
        np.where(df['American'] == True,'American',
        np.where(df['Latin American'] == True,'Latin American',
        np.where(df['FrenchEuropean'] == True,'French/European',
        np.where(df['Middle Eastern'] == True,'Middle Eastern',
        np.where(df['Southern European'] == True,'Southern European',
        np.where(df['BritishIrish'] == True,'Brisith/Irish',
        np.where(df['African'] == True,'African',
        np.where(df['Bar'] == True,'Bar/Drinks','Other'))))))))))))

# Check if restaurants are marked to have vegan, vega or gluten free options and if they're for breakfast or lunch
df['Vegan'] = df['category'].str.contains('Vegan')
df['Gluten Free'] = df['category'].str.contains('Gluten Free')
df['Vegetarian'] = df['category'].str.contains('Vegetarian')
df['Breakfast'] = df['category'].str.contains('Breakfast')
df['Lunch'] = df['category'].str.contains('Lunch')

In [170]:
# Get the smaller version (with no yes-no cuisine colunns)
df_new = pd.concat([df.iloc[:,0:16],df.iloc[:,28:]], axis = 1)
df_new.to_csv('Tripadvisor_cuisine_Liverpool.csv')
df_new

,title,text,review date,rating,restaurant,location,total reviews,1-star,2-star,3-star,...,float rating,category,avg_rating,price,cuisine,Vegan,Gluten Free,Vegetarian,Breakfast,Lunch
0,Holiday,very busy but sorry to say the mens toliets we...,22 January 2023,3.0,Rudy's Albert Dock,"Unit 6, Britannia Pavillion, Liverpool L3 4AD ...",218,0,3,5,...,4.78,['Italian' 'Pizza'],5.0,NaN,Italian,False,False,False,False,False
1,Delicious,To be honest I’m not a pizza lover but my husb...,22 January 2023,5.0,Rudy's Albert Dock,"Unit 6, Britannia Pavillion, Liverpool L3 4AD ...",218,0,3,5,...,4.78,['Italian' 'Pizza'],5.0,NaN,Italian,False,False,False,False,False
2,Great pizza great value great service,"Amazing pizzas, with lots of choice. Excellenc...",22 January 2023,5.0,Rudy's Albert Dock,"Unit 6, Britannia Pavillion, Liverpool L3 4AD ...",218,0,3,5,...,4.78,['Italian' 'Pizza'],5.0,NaN,Italian,False,False,False,False,False
3,Was so looking forward to it but........,Booked. Arrived. Nice place in Albert Dock. Re...,22 January 2023,2.0,Rudy's Albert Dock,"Unit 6, Britannia Pavillion, Liverpool L3 4AD ...",218,0,3,5,...,4.78,['Italian' 'Pizza'],5.0,NaN,Italian,False,False,False,False,False
4,1st time visit to Rudy's Albert Dock,"Great location, great service, great atmospher...",21 January 2023,5.0,Rudy's Albert Dock,"Unit 6, Britannia Pavillion, Liverpool L3 4AD ...",218,0,3,5,...,4.78,['Italian' 'Pizza'],5.0,NaN,Italian,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Disappointed,After looking forward to a bite to eat with fr...,12 January 2023,1.0,Mb Cafe 62,"62 Muirhead Avenue East, Liverpool L11 1EL Eng...",5,1,0,0,...,4.20,['Mediterranean' 'British' 'Turkish'],4.0,NaN,Middle Eastern,False,False,False,False,False
1,Great breakfast,This is the second time we have been and the f...,2 January 2023,5.0,Mb Cafe 62,"62 Muirhead Avenue East, Liverpool L11 1EL Eng...",5,1,0,0,...,4.20,['Mediterranean' 'British' 'Turkish'],4.0,NaN,Middle Eastern,False,False,False,False,False
2,First try of a local gem,"We visited today, it was busy with lots of dif...",30 December 2022,5.0,Mb Cafe 62,"62 Muirhead Avenue East, Liverpool L11 1EL Eng...",5,1,0,0,...,4.20,['Mediterranean' 'British' 'Turkish'],4.0,NaN,Middle Eastern,False,False,False,False,False
3,Absolutely worth a visit,I’ve been several times now - for lunch as wel...,17 September 2022,5.0,Mb Cafe 62,"62 Muirhead Avenue East, Liverpool L11 1EL Eng...",5,1,0,0,...,4.20,['Mediterranean' 'British' 'Turkish'],4.0,NaN,Middle Eastern,False,False,False,False,False


In [171]:
# Check the uncategorized cuisines
df[df['cuisine'] == 'Other']['category'].unique()

array(["['Fast food' 'International']"], dtype=object)

In [157]:
# Quit the Chrome driver
driver.quit()

In [73]:
def get_reviews(driver):
    i=1
    review_master = pd.DataFrame()
    while i <= 20: # either breaks before or at 20 pages of reviews 15*20 = 300 in total.
        try: # if the button "...More" is available click it.
            driver.find_element(By.XPATH,"//*[@class ='taLnk ulBlueLinks']").click()
        except: # otherwise the texts are short enough not needing such a button
            pass
        time.sleep(2)
        review = extract_reviews() # Get the reviews itself, see below
        review_master = pd.concat([review_master, review], axis = 0) # combine with already scraped reviews
        try: # navigate to the next page
            driver.find_elements(By.XPATH,"*//a[@class = 'nav next ui_button primary']")[0].click() 
            print('page',i)
            time.sleep(2)
        except:# Otherwise its the last page, it will tell then break.
            print('page',i,'is last')
            break
        i=i+1
    return(review_master)

In [11]:
def extract_reviews():
    reviews = pd.DataFrame()
    reviews['title'] = [txt.text for txt in driver.find_elements(By.CLASS_NAME,"noQuotes")]
        
    splitted = [txt.text.split('\n') for txt in driver.find_elements(By.XPATH,"//div[contains(@class, 'ui_column is-9')]")[:15]]
    
    # We don't want answers from the restaurant, translations or others.
    revtxt = [np.array(split)[[spl != 'via mobile' for spl in split]][2] for split in splitted]
    reviews['text'] = np.array(revtxt)[[re.match('WHAT IS YOUR ANSWER|Answer|^AnswerShow|Google Translation',\
                                                 txt) == None for txt in revtxt]].tolist()
    
    reviews['review date'] = [txt.get_attribute("title") for txt in driver.find_elements(By.CLASS_NAME,"ratingDate")]

    # Get the bubble ratings elements value (rating)
    RevB = driver.find_elements(By.XPATH,"*//div[@class ='ui_column is-9']//span[contains(@class, 'ui_bubble_rating bubble_')]")

    reviews['rating'] = [int(re.findall('[0-9]+',txt.get_attribute("class"))[0])/10 for txt in RevB]
    
    try: # the restaurant is not at the expected place in very rare cases.
        reviews['restaurant'] = driver.find_element(By.CLASS_NAME,"HjBfq").text
    except:
        reviews['restaurant'] = driver.find_elements(By.CLASS_NAME,"breadcrumb")[-1].text
        
    reviews['location'] = [el.text for el in driver.find_elements(By.CLASS_NAME,"yEWoV")][0]
    reviews['total reviews'] = int(driver.find_elements(By.CLASS_NAME,"IcelI")[0].text.split(' ')[0]\
                                   .replace(',','').replace('.',''))
    
    # The cuisines, prices and extras may be in a different place. Can be these four.
    elementD = [txt.text for txt in driver.find_elements(By.CLASS_NAME,"dlMOJ")]
    elementS = [txt.text for txt in driver.find_elements(By.CLASS_NAME,"SrqKb")]
    elementA = [txt.text for txt in driver.find_elements(By.CLASS_NAME,"AGRBq")]
    elementR = [txt.text for txt in driver.find_elements(By.CLASS_NAME,"RMOVd")]
    
    if len(np.array(elementD)[[re.match('[€,£,$]',txt) != None for txt in elementD]])>0:
        reviews['price'] = np.array(elementD)[[re.match('[€,£,$]',txt) != None for txt in elementD]][0]
    
    elif len(np.array(elementS)[[re.match('[€,£,$]',txt) != None for txt in elementS]])>0:
        reviews['price'] = np.array(elementS)[[re.match('[€,£,$]',txt) != None for txt in elementS]][0]
        
    elif len(np.array(elementA)[[re.match('[€,£,$]',txt) != None for txt in elementA]])>0:
        reviews['price'] = np.array(elementA)[[re.match('[€,£,$]',txt) != None for txt in elementA]][0]
    
    else: 'unknown'
                                        
    checkbox = [txt.text for txt in driver.find_elements(By.XPATH,"*//div[@class = 'ui_checkbox item']")]
    
    reviews['1-star'] = int(np.array(checkbox)[[re.match('Terrible',check) != None for check in checkbox]][0].split(' ')[-1]\
                                   .replace(',','').replace('.',''))
    reviews['2-star'] = int(np.array(checkbox)[[re.match('Poor',check) != None for check in checkbox]][0].split(' ')[-1]\
                                   .replace(',','').replace('.',''))
    reviews['3-star'] = int(np.array(checkbox)[[re.match('Average',check) != None for check in checkbox]][0].split(' ')[-1]\
                                   .replace(',','').replace('.',''))
    reviews['4-star'] = int(np.array(checkbox)[[re.match('Very good',check) != None for check in checkbox]][0].split(' ')[-1]\
                                   .replace(',','').replace('.',''))
    reviews['5-star'] = int(np.array(checkbox)[[re.match('Excellent',check) != None for check in checkbox]][0].split(' ')[-1]\
                                   .replace(',','').replace('.',''))
    reviews['float rating']   = round((reviews['1-star']*1+reviews['2-star']*2+reviews['3-star']*3+\
                                reviews['4-star']*4+reviews['5-star']*5)/reviews['total reviews'],2)
    
    if len(np.array(elementD)[[re.match('€',txt) == None for txt in elementD]])>0:
        reviews['category'] = np.array2string(np.array(elementD)\
                 [[re.match('[€,£,$]',txt) == None for txt in elementD]])
    
    elif len(np.array(elementS)[[re.match('€',txt) == None for txt in elementS]])>0:
        reviews['category'] = np.array(elementS)\
                 [[re.match('[€,£,$]',txt) == None for txt in elementS]][0]
    
    elif len(np.array(elementR)[[re.match('€',txt) == None for txt in elementR]])>0:
        reviews['category'] = np.array(elementR)\
                 [[re.match('[€,£,$]',txt) == None for txt in elementR]][0]
        
    elif len(np.array(elementA)[[re.match('€',txt) == None for txt in elementA]])>0:
        reviews['category'] = np.array2string(np.array(elementA[:-1])\
                 [[re.match('[€,£,$]',txt) == None for txt in elementA[:-1]]])
    else:
        reviews['category'] = 'unknown'
        
    RevA = driver.find_elements(By.XPATH,"*//span[@class ='MFMAn']//span[contains(@class, 'ui_bubble_rating bubble_')]")

    reviews['avg_rating'] = int(re.findall('[0-9]+',RevA[0].get_attribute("class"))[0])/10
    return(reviews)